# Multi-Query Code

Multi-Query là một phương pháp nâng cao của giai đoạn Query Transformation (Biến đổi truy vấn) trong retrieval (truy xuất thông tin).

Trong retrieval truyền thống, bạn thường cung cấp 1 câu hỏi hoặc query (truy vấn) cho database (cơ sở dữ liệu) của mình và sử dụng một similarity measure (đo lường độ tương đồng), bạn sẽ nhận lại 4-5 documents (tài liệu).

Tuy nhiên, phương pháp multi-query (đa truy vấn) sẽ tạo ra nhiều queries (truy vấn) (đúng như tên gọi) và sau đó sử dụng từng query (truy vấn) để lấy các docs (tài liệu) tương tự.

Mục tiêu là các docs (tài liệu) được trả về đại diện cho một context (ngữ cảnh) toàn diện hơn để LLM (mô hình ngôn ngữ lớn) của bạn làm việc.

![](https://retrieval-tutorials.vercel.app/assets/images/MultiQuery-44880700b3e328b054c4c1f93dcf7e49.gif)

### Tại sao điều này hữu ích?

Thông thường, các nhà phát triển sẽ sử dụng Multi-Query vì hai lý do chính: Tăng cường một query (truy vấn) không tối ưu và Mở rộng tập kết quả.

Tăng cường một query (truy vấn) không tối ưu

Người dùng không phải lúc nào cũng đưa ra các queries (truy vấn) tốt nhất, chúng ta không thể trách họ được - Họ chỉ đang cố gắng sử dụng sản phẩm của bạn, chứ không phải xây dựng một query (truy vấn) hoàn hảo.

Để giúp giải quyết vấn đề này, chúng ta chuyển sang phương pháp multi-query (đa truy vấn) để giúp chúng ta lấp đầy bất kỳ khoảng trống nào trong query (truy vấn) của người dùng.

Mở rộng tập kết quả

Với nhiều queries (truy vấn), bạn có khả năng nhận được nhiều kết quả hơn từ database (cơ sở dữ liệu) của mình. Mục tiêu của multi-query (đa truy vấn) là có một tập kết quả mở rộng, có thể trả lời các câu hỏi tốt hơn so với các docs (tài liệu) từ một query (truy vấn) duy nhất.

Các kết quả này sẽ được deduplicated (loại bỏ trùng lặp - trong trường hợp cùng một document (tài liệu) được trả về nhiều lần) và sau đó được sử dụng làm context (ngữ cảnh) trong prompt (lời nhắc) cuối cùng của bạn.


In [4]:
from dotenv import load_dotenv
import os

load_dotenv(dotenv_path='.env')

True

In [5]:
import re
import pymupdf4llm

md_text = pymupdf4llm.to_markdown(
    doc='data/vneid.pdf', 
    write_images=True,
    embed_images=False,
    image_path="data/md_img",)


def preprocess_markdown(markdown_text):
    # Thay thế các dòng '# Hình ...' thành văn bản không phải tiêu đề
    processed_text = re.sub(r'^# (Hình .+)$', r'\1', markdown_text, flags=re.MULTILINE)
    processed_text = processed_text.replace('\t', ' ')
    return processed_text

cleaned_markdown = preprocess_markdown(md_text)

Processing data/vneid.pdf...
[                                        ] (0/13[                                        ] (  1/13[                                        ] (  2/13[                                        ] (  3/136[=                                       ] (  4/1[=                                       ] (  5/1[=                                       ] (  6/13[==                                      ] (  7/13[==                                      ] (  8/13[==                                      ] (  9/13[==                                      ] ( 10/136[===                                     ] ( 11/1[===                                     ] ( 12/1[===                                     ] ( 13/13[====                                    ] ( 14/13[====                                    ] ( 15/13[====                                    ] ( 16/136[=====                                   ] ( 17/136[=====                                   ] ( 18/1[=====                  

In [6]:
import pathlib
pathlib.Path("data/vneid.md").write_bytes(cleaned_markdown.encode())

80209

In [7]:
from langchain_ollama.embeddings import OllamaEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import  FAISS
from langchain_community.document_loaders import UnstructuredMarkdownLoader

loader = UnstructuredMarkdownLoader(
    "data/vneid.md",
    mode="elements",
    strategy="fast",
)

documents = loader.load()

sentence_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
    separators=["\n\n", "\n", " "]
)

split_sentences = sentence_text_splitter.split_documents(documents)
embeddings = OllamaEmbeddings(model='bge-m3:latest')
vectordb = FAISS.from_documents(split_sentences, embeddings)

In [8]:
# Setp multi query
from langchain_ollama import ChatOllama
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.prompts import PromptTemplate
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [12]:
question = 'Cách đăng ký vneid mức O'
llm = ChatOllama(model='gemma3:4b', temperature=0)

retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=vectordb.as_retriever(),
    llm=llm
)

Sau đó, khi chúng ta thực sự yêu cầu các tài liệu liên quan, chúng ta sẽ thấy nhiều câu hỏi khác được tạo ra từ câu hỏi gốc của chúng ta.

In [13]:
unique_docs = retriever_from_llm.invoke(question)
unique_docs

INFO:langchain.retrievers.multi_query:Generated queries: ['How do I register for VNeID Level O?', 'What are the steps involved in registering for VNeID Tier 0?', 'I want to sign up for VNeID at the basic level – how do I do that?']


[Document(id='803d968f-c309-4f84-a27e-b8a9d015695b', metadata={'source': 'data/vneid.md', 'category_depth': 0, 'languages': ['vie'], 'file_directory': 'data', 'filename': 'vneid.md', 'filetype': 'text/markdown', 'last_modified': '2025-03-31T13:53:22', 'category': 'Title', 'element_id': '7108b7d7271cb4632f9d0abaedf0414e'}, page_content='dụng VNeID'),
 Document(id='465b83d2-db5c-43a6-b03a-878f3f435fed', metadata={'source': 'data/vneid.md', 'category_depth': 0, 'languages': ['vie'], 'file_directory': 'data', 'filename': 'vneid.md', 'filetype': 'text/markdown', 'last_modified': '2025-03-31T13:53:22', 'category': 'Title', 'element_id': '5c876da1e5a02c52c35a2118839cbada'}, page_content='dụng VNeID'),
 Document(id='a1490aa2-c5bf-4572-81e6-a5dd259829c5', metadata={'source': 'data/vneid.md', 'category_depth': 0, 'languages': ['vie'], 'file_directory': 'data', 'filename': 'vneid.md', 'filetype': 'text/markdown', 'last_modified': '2025-03-31T13:53:22', 'category': 'Title', 'element_id': '010e5633

In [14]:
unique_docs[3].page_content

'Bước 1: NSD ấn nút “Đăng nhập” trên giao diện giới thiệu ứng dụng VNeID. Hệ thống hiển thị màn hình Đăng nhập ứng dụng'

# Chỉnh sửa Prompt để tạo thêm Queries 

Nếu bạn muốn chỉnh sửa prompt template (mẫu lời nhắc) đang được MultiQuery Retriever sử dụng, bạn có thể làm điều đó khi bạn tạo nó lần đầu tiên. Xem prompt (lời nhắc) gốc đang được sử dụng ở đây.


In [15]:
prompt_template = """You are an AI language model assistant.

Your task is to generate 3 different versions of the given user question in Vietnamese to retrieve relevant documents from a vector database.

By generating multiple perspectives on the user question, your goal is to help the user overcome some of the limitations  of distance-based similarity search.

Provide these alternative questions separated by newlines.

Original question: {question}"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["question"]
)

retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=vectordb.as_retriever(), llm=llm, prompt=PROMPT
)

In [16]:
unique_docs = retriever_from_llm.invoke(question)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. Hướng dẫn đăng ký tài khoản VNeID cấp độ cơ bản.', '2. Quy trình đăng ký VNeID mức tối thiểu.', '3. Chi tiết về việc đăng ký VNeID cấp O – yêu cầu và thủ tục.']


In [17]:
unique_docs

[Document(id='ddf14239-ffa6-4b5b-bc4b-a25c7379b353', metadata={'source': 'data/vneid.md', 'languages': ['vie'], 'file_directory': 'data', 'filename': 'vneid.md', 'filetype': 'text/markdown', 'last_modified': '2025-03-31T13:53:22', 'parent_id': '472ac5ec2c06e78521e66518241e9052', 'category': 'UncategorizedText', 'element_id': '4b55048c9ee4c15e0ea7dd34b21fcdd7'}, page_content='Bước 1: NSD có thể kích hoạt bằng các cách chọn kích hoạt tài khoản trên Trang giới thiệu ứng dụng VNeID'),
 Document(id='af6d08f8-38e8-4309-9960-3ec57d7b0c8f', metadata={'source': 'data/vneid.md', 'category_depth': 1, 'languages': ['vie'], 'file_directory': 'data', 'filename': 'vneid.md', 'filetype': 'text/markdown', 'last_modified': '2025-03-31T13:53:22', 'parent_id': 'd0328e758a8f268624f3bba7a0e044ec', 'category': 'ListItem', 'element_id': 'd99a96a66e22174d26c1fb4b162fe280'}, page_content='Bước 1: NSD đăng nhập vào ứng dụng VNeID trên thiết bị di động.'),
 Document(id='ed0c44ee-0d52-4a8b-bfe9-9c7217b891b0', meta

In [20]:
for doc in unique_docs:
    print("\n=================DOCUMENT=================\n")
    print(doc.page_content)
    print('\n')
    print(doc.metadata)


=================DOCUMENT=================

Bước 1: NSD có thể kích hoạt bằng các cách chọn kích hoạt tài khoản trên Trang giới thiệu ứng dụng VNeID


{'source': 'data/vneid.md', 'languages': ['vie'], 'file_directory': 'data', 'filename': 'vneid.md', 'filetype': 'text/markdown', 'last_modified': '2025-03-31T13:53:22', 'parent_id': '472ac5ec2c06e78521e66518241e9052', 'category': 'UncategorizedText', 'element_id': '4b55048c9ee4c15e0ea7dd34b21fcdd7'}

=================DOCUMENT=================

Bước 1: NSD đăng nhập vào ứng dụng VNeID trên thiết bị di động.


{'source': 'data/vneid.md', 'category_depth': 1, 'languages': ['vie'], 'file_directory': 'data', 'filename': 'vneid.md', 'filetype': 'text/markdown', 'last_modified': '2025-03-31T13:53:22', 'parent_id': 'd0328e758a8f268624f3bba7a0e044ec', 'category': 'ListItem', 'element_id': 'd99a96a66e22174d26c1fb4b162fe280'}

=================DOCUMENT=================

Bước 1: NSD ấn nút “Đăng nhập” trên giao diện giới thiệu ứng dụng VNeID. Hệ th